# [모듈 1.1] CodeCommit 리파지토리 생성

이 노트북은 CodeCommit 리파지토리를 생성하고, codecommit 폴더를 리파지토리에 Push 하는 노트북 입니다.

## 참고 자료
- [Quick guide to start using AWS CodeCommit via AWS CLI](https://dev.to/tiamatt/quick-guide-to-start-using-aws-codecommit-via-aws-cli-1jg5)
- 개발자 가이드
    - [Setup for HTTPS users using Git credentials](https://docs.aws.amazon.com/codecommit/latest/userguide/setting-up-gc.html)
    - [Create an AWS CodeCommit repository](https://docs.aws.amazon.com/codecommit/latest/userguide/how-to-create-repository.html#how-to-create-repository-cli)
    



# 1. 환경 확인

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

In [2]:
%%sh 
git --version

git version 2.37.1


In [3]:
%%sh
aws --version

aws-cli/1.27.20 Python/3.8.12 Linux/5.10.102-99.473.amzn2.x86_64 botocore/1.29.32


# 2. 파라미터 로딩

In [4]:
%store -r code_pipeline_train_config_json_path

from common_utils import load_json

code_pipeline_train_dict = load_json(code_pipeline_train_config_json_path)

import json
print (json.dumps(code_pipeline_train_dict, indent=2))

code_pipeline_train_repository_name = code_pipeline_train_dict["train_code_repo_name"]

{
  "code_pipeline_role_arn": "arn:aws:iam::057716757052:role/MLOps-CodePipelineRole",
  "code_build_service_arn": "arn:aws:iam::057716757052:role/MLOps-CodeBuildRole",
  "project_prefix": "CodePipeline-Train-NCF",
  "region": "ap-northeast-2",
  "account_id": "057716757052",
  "train_code_repo_name": "ncf-training-repo",
  "code_build_project_name": "ncf-code-build-training-sm-pipeline",
  "bucket": "sagemaker-ap-northeast-2-057716757052",
  "code_pipeline_name": "ncf-code-pipeline-training-sm-pipeline",
  "model_package_group_name": "NCF-Model-CodePipeline",
  "branch_name": "master",
  "update_time": "2022-12-22-07-34-25"
}


In [5]:

source_code = 'codecommit'


# 3.리파지토리 생성

In [6]:
%%sh -s {code_pipeline_train_repository_name}
repository_name=$1
aws codecommit create-repository --repository-name $repository_name \
--repository-description "My demonstration repository" 

{
    "repositoryMetadata": {
        "accountId": "057716757052",
        "repositoryId": "0f40762f-8760-4d52-bc87-270be4ad17bd",
        "repositoryName": "ncf-training-repo",
        "repositoryDescription": "My demonstration repository",
        "lastModifiedDate": 1671694623.141,
        "creationDate": 1671694623.141,
        "cloneUrlHttp": "https://git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/ncf-training-repo",
        "cloneUrlSsh": "ssh://git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/ncf-training-repo",
        "Arn": "arn:aws:codecommit:ap-northeast-2:057716757052:ncf-training-repo"
    }
}


# 4. 리파지토리 로컬에 클로닝


In [7]:
import boto3

clone_url = boto3.client('codecommit').get_repository(repositoryName = code_pipeline_train_repository_name).get('repositoryMetadata').get('cloneUrlHttp')
print(clone_url)


https://git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/ncf-training-repo


In [8]:
%%sh -s {clone_url} {code_pipeline_train_repository_name}

clone_url=$1
repository_name=$2

rm -rf $repository_name

git clone $clone_url 

Cloning into 'ncf-training-repo'...


# 5. 소스 코드 복사

In [9]:
%%sh -s {code_pipeline_train_repository_name} {source_code}
repository_name=$1
source_code=$2

cp -r $source_code/* $repository_name
ls $repository_name

codebuild-buildspec.yml
CONTRIBUTING.md
img
LICENSE
origin-sagemaker-pipelines-project.ipynb
pipelines
README.md
sagemaker-pipelines-project.ipynb
setup.cfg
setup.py


# 6. 코드 파일을 리파지토리에 푸시

In [10]:
%%sh -s {code_pipeline_train_repository_name} 
repository_name=$1
cd $repository_name
echo $PWD
git add .
git commit -m"Add new files"
git push origin master


/home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/4_sm-train-codepipeline/ncf-training-repo
[master (root-commit) 9985c28] Add new files
 Committer: EC2 Default User <ec2-user@ip-172-16-26-229.ap-northeast-2.compute.internal>
Your name and email address were configured automatically based
on your username and hostname. Please check that they are accurate.
You can suppress this message by setting them explicitly:

    git config --global user.name "Your Name"
    git config --global user.email you@example.com

After doing this, you may fix the identity used for this commit with:

    git commit --amend --reset-author

 39 files changed, 4473 insertions(+)
 create mode 100644 CONTRIBUTING.md
 create mode 100644 LICENSE
 create mode 100644 README.md
 create mode 100644 codebuild-buildspec.yml
 create mode 100644 img/pipeline-full.png
 create mode 100644 origin-sagemaker-pipelines-project.ipynb
 create mode 100644 pipelines/.ipynb_checkpoints/run_pipeline-checkpo

remote: Validating objects: 100%        
To https://git-codecommit.ap-northeast-2.amazonaws.com/v1/repos/ncf-training-repo
 * [new branch]      master -> master


# 7. 변수 저장

In [11]:
training_repository_name = code_pipeline_train_repository_name
%store training_repository_name

Stored 'training_repository_name' (str)


# 8. Code Commit 생성 확인
- Code Commit 콘솔에서 확인 하세요.

![ncf_code_commit.png](img/ncf_code_commit.png)